# Instructions
The following code was designed in order to calculate the fluorescent intensity for either an entire image or a region of interest (ROI) within an image based upon a user-made mask.  When an ROI is used, the size of the ROI in pixels is returned. When an ROI is not used, ROI size is set to size of image.


### Folder Structure

For this code to work files should be organized in a specific manner.
Directory_Main (the path of which is to be defined by the user... actual directory name can be whatever you want) should contain the subfolders "ROI", and "SavedOutput" (these names matter).  Additionally, there should be a folder with the images to measured.  The name of this folder can be specified by user
If you are not using ROIs, this folder can be excluded.
Image folder should contain .tif images. ROI should contain .tif images with ROI masks, described below.  These should be named similarly to files in image folder with the exception of some suffix.
Example filenames under image folder and ROI: "Mouse1_Image1_Ch1.tif",""Mouse1_Image1_ROI.tif"

### Making ROIs
Regions of interest are best drawn using ImajeJ.  Binary masks are created such that the region of interest pixel values = 255, whereas other values are equal to 0
#1 - Open the image in ImageJ
#2 - Use your preferred selection tool (I like freehand selection tool) to outline the region of interest
#3 - Go to edit > selection > create mask
#4 - Save the mask as .tif file

### Requirements
The following will need to be installed in your Conda environment:

1 - python (3.6.5)

2 - jupyter

3 - imread

4 - mahotas(1.4.4)

5 - numpy(1.14.3)

6 - pandas(0.23.0)

7 - matplotlib(2.2.2). 

The following commands can be executed in your terminal to create the environment: 

conda config --add channels conda-forge

conda create -n EnvironmentName python=3.6.5 mahotas=1.4.4 pandas=0.23.0 matplotlib=2.2.2 jupyter imread

Different versions of these packages have not been tested.  Single channel .tif images are required (some image acquisition software exports single channel images with more than >2 dimensions.  If this is the case, images must first be converted to 2 dimensional space).  Images that are not already 8-bit are converted to 8 bit.  However, these images are not necessarily on a scale from 0-255.  It is therefore advised that all images are set to 8 bit before running this. 


# Specify Parameters and Options for Running
### This should be the only part of the code that is required to be changed

In [ ]:
# Define directory of working environment (Directory.Main)
Directory_Main = "/Users/zachpennington/Documents/ZachsDocuments/Sinai.PostDoc/Projects/Linking/RetrospectiveLinking/RL3_HM4Da/RL3_Images"
Image_Folder = "Ch1" #name of folder within Directory_Main that contains images to be measured

#Options (set to 0 or 1)
UseROI = 1 #Only count cells within a region of interest?

# Load Necessary Packages

In [ ]:
#Note that this code was written in python 3.6.5
import pylab 
import os
import fnmatch
import imread
import numpy as np
import mahotas as mh 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# Get Directory Information

In [ ]:
#Define existing subdirectories
Directory_Images = Directory_Main + "/" + Image_Folder
Directory_ROI = Directory_Main + "/" + "ROI"
Directory_Output = Directory_Main + "/" + "SavedOutput"


#Get filenames and create output subdirectories based upon usage
FileNames_Images = sorted(os.listdir(Directory_Images))
FileNames_Images = fnmatch.filter(FileNames_Images, '*.tif') #restrict files to .tif images

#Create output directory if it is not in existence already
try:
    os.mkdir(Directory_Output)
except FileExistsError: 
    pass

#Get ROI informatiohn
if UseROI == 1:
    FileNames_ROI = sorted(os.listdir(Directory_ROI))
    FileNames_ROI = fnmatch.filter(FileNames_ROI, '*.tif') #restrict files to .tif images

# Measure Intensity

In [ ]:
#Initialize arrays to store data in
Intensity = []
ROI_SIZE = []
    
#Loop through images and count cells
for x, File in enumerate(FileNames_Images):
    
    #Load Image
    print("Processing: " + File)
    Image_Current_Gray = mh.imread((Directory_Images + "/" + File),as_grey=True) #Load File as greyscale image
     
    #Measure Intensity
    if UseROI:
        ROI_Current = mh.imread((Directory_ROI + "/" + FileNames_ROI[x]),as_grey=True) #Load File
        Intensity.append(np.average(Image_Current_Gray[ROI_Current!=0])) #Calculate average intensity in specified ROI
        ROI_SIZE.append(np.count_nonzero(ROI_Current))
    else:
        Intensity.append(np.average(Image_Current_Gray))
        ROI_SIZE.append(Image_Current_Gray.size)    

# Write Data to Disk

In [ ]:
#Create pandas DataFrame and write to disk
DataFrame = pd.DataFrame(
    {'FileNames': FileNames_Images,
     'UseROI' : np.ones(len(FileNames_Images))*UseROI,
     'GrayIntensity': Intensity,
     'ROIsize': ROI_SIZE
    })
DataFrame.to_csv(Directory_Output + "/" + Image_Folder + "_Intensity.csv")